# Carteras multifactoriales

## Aproximación Top-Down

Objetivo

Construir una cartera long-only que tenga:

- Exposición positiva a Value y Momentum

- Beta próxima a la de mercado

- Volatilidad objetivo 12% anual

Paso 1. Definir “mix” de factores (objetivos)

Decides ex ante las exposiciones:
$$\beta^*_{value}=0.4,\beta^*_{mom}=0.3,\beta^*_{MKT}=1 $$

Interpretación: “Quiero un tilt fuerte a value, moderado a momentum; no quiero que el resultado sea simplemente más beta o un sector”.

In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp
import datetime as dt
import yfinance as yf


## 1 Obtenemos los datos de los activos y los depuramos

In [ ]:
ibex35_tickers = [
    "ACS.MC","ACX.MC","AMS.MC","ANA.MC","ANE.MC",
    "AENA.MC","BBVA.MC","BKT.MC","CABK.MC","CLNX.MC",
    "COL.MC","ELE.MC","ENG.MC","FDR.MC","FER.MC",
    "GRF.MC","IAG.MC","IBE.MC","IDR.MC","ITX.MC",
    "LOG.MC","MAP.MC","MRL.MC","MTS.MC","NTGY.MC",
    "PUIG.MC","RED.MC","REP.MC","ROVI.MC","SAB.MC",
    "SAN.MC","SCYR.MC","SLR.MC","TEF.MC","UNI.MC"
]

ibex_index = "^IBEX"


In [ ]:
# Últimos 5 años (aprox.)
end = dt.datetime.today()
start = end - dt.timedelta(days=5*365)

# Descarga diaria (auto-adjust) y paso a mensual (último cierre de cada mes)
px_daily = yf.download(
    ibex35_tickers,
    start=start.strftime("%Y-%m-%d"),
    end=end.strftime("%Y-%m-%d"),
    auto_adjust=True,
    progress=False
)["Close"]

px_monthly = px_daily.resample("M").last()

# Retornos mensuales (simples)
rets_monthly = px_monthly.pct_change().dropna(how="all")

print("Rango mensual:", px_monthly.index.min().date(), "->", px_monthly.index.max().date())
print("Shape precios:", px_monthly.shape, "| Shape retornos:", rets_monthly.shape)

px_monthly.head()

Rango mensual: 2021-01-31 -> 2026-01-31
Shape precios: (61, 35) | Shape retornos: (60, 35)


/tmp/ipython-input-2161981578.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  px_monthly = px_daily.resample("M").last()


Ticker,ACS.MC,ACX.MC,AENA.MC,AMS.MC,ANA.MC,ANE.MC,BBVA.MC,BKT.MC,CABK.MC,CLNX.MC,...,PUIG.MC,RED.MC,REP.MC,ROVI.MC,SAB.MC,SAN.MC,SCYR.MC,SLR.MC,TEF.MC,UNI.MC
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-31,18.907900,6.923791,125.800224,49.508205,105.200439,NaN,2.819356,2.486552,1.552799,43.892067,...,NaN,11.810524,5.992204,37.337135,0.279963,2.022224,1.421884,21.240,2.414027,0.443355
2021-02-28,18.628546,7.272244,139.239990,54.181984,113.449783,NaN,3.445048,2.976777,1.790890,40.930191,...,NaN,10.456763,7.683920,41.567448,0.322368,2.426250,1.645232,18.600,2.395008,0.563705
2021-03-31,20.775166,8.390760,136.670639,56.895802,121.529053,NaN,3.312575,3.208760,1.963506,44.609821,...,NaN,11.388180,7.790898,42.303162,0.352856,2.425831,1.716012,18.080,2.592668,0.665809
2021-04-30,19.944450,8.707535,142.995224,53.428150,123.059853,NaN,3.542232,3.429996,1.984339,46.141327,...,NaN,11.520161,7.332002,44.326351,0.408105,2.715462,1.813530,17.055,2.617460,0.642792
2021-05-31,18.746166,8.850839,141.809372,58.177322,117.446899,NaN,3.895280,3.530102,2.107914,48.142357,...,NaN,12.383704,8.059445,52.419128,0.485022,2.891989,1.736459,15.940,2.730554,0.713691


In [ ]:
# Retornos mensuales (simples)
rets_monthly = px_monthly.pct_change().dropna(how="all")

print("Rango mensual:", px_monthly.index.min().date(), "->", px_monthly.index.max().date())
print("Shape precios:", px_monthly.shape, "| Shape retornos:", rets_monthly.shape)

px_monthly.head()

Rango mensual: 2021-01-31 -> 2026-01-31
Shape precios: (61, 35) | Shape retornos: (60, 35)


Ticker,ACS.MC,ACX.MC,AENA.MC,AMS.MC,ANA.MC,ANE.MC,BBVA.MC,BKT.MC,CABK.MC,CLNX.MC,...,PUIG.MC,RED.MC,REP.MC,ROVI.MC,SAB.MC,SAN.MC,SCYR.MC,SLR.MC,TEF.MC,UNI.MC
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-31,18.907900,6.923791,125.800224,49.508205,105.200439,NaN,2.819356,2.486552,1.552799,43.892067,...,NaN,11.810524,5.992204,37.337135,0.279963,2.022224,1.421884,21.240,2.414027,0.443355
2021-02-28,18.628546,7.272244,139.239990,54.181984,113.449783,NaN,3.445048,2.976777,1.790890,40.930191,...,NaN,10.456763,7.683920,41.567448,0.322368,2.426250,1.645232,18.600,2.395008,0.563705
2021-03-31,20.775166,8.390760,136.670639,56.895802,121.529053,NaN,3.312575,3.208760,1.963506,44.609821,...,NaN,11.388180,7.790898,42.303162,0.352856,2.425831,1.716012,18.080,2.592668,0.665809
2021-04-30,19.944450,8.707535,142.995224,53.428150,123.059853,NaN,3.542232,3.429996,1.984339,46.141327,...,NaN,11.520161,7.332002,44.326351,0.408105,2.715462,1.813530,17.055,2.617460,0.642792
2021-05-31,18.746166,8.850839,141.809372,58.177322,117.446899,NaN,3.895280,3.530102,2.107914,48.142357,...,NaN,12.383704,8.059445,52.419128,0.485022,2.891989,1.736459,15.940,2.730554,0.713691


In [ ]:
def clean_monthly_prices(px_monthly: pd.DataFrame,
                         min_coverage: float = 0.80,
                         fill_method: str | None = "ffill",
                         max_fill_gap: int = 2) -> pd.DataFrame:
    """
    Limpieza de precios mensuales:
    - ordena índice y elimina duplicados
    - elimina filas y columnas totalmente NaN
    - elimina activos con cobertura < min_coverage
    - rellena gaps pequeños (opcional) con ffill limitado a max_fill_gap
    """
    px = px_monthly.copy()

    # 1) índice ordenado y sin duplicados
    px = px.sort_index()
    px = px[~px.index.duplicated(keep="last")]

    # 2) quitar filas/columnas completamente vacías
    px = px.dropna(how="all", axis=0)
    px = px.dropna(how="all", axis=1)

    # 3) filtrar por cobertura mínima (por activo)
    coverage = px.notna().mean(axis=0)
    keep_cols = coverage[coverage >= min_coverage].index
    px = px[keep_cols]

    # 4) rellenar gaps pequeños (opcional)
    if fill_method == "ffill":
        px = px.ffill(limit=max_fill_gap)
    elif fill_method == "bfill":
        px = px.bfill(limit=max_fill_gap)
    elif fill_method is None:
        pass
    else:
        raise ValueError("fill_method debe ser 'ffill', 'bfill' o None")

    # 5) tras el fill, quitar filas que siguen siendo todas NaN (por seguridad)
    px = px.dropna(how="all", axis=0)

    return px

px_clean = clean_monthly_prices(px_monthly, min_coverage=0.80, fill_method="ffill", max_fill_gap=2)

print("Precios (raw):", px_monthly.shape, "-> (clean):", px_clean.shape)
px_clean


Precios (raw): (61, 35) -> (clean): (61, 34)


Ticker,ACS.MC,ACX.MC,AENA.MC,AMS.MC,ANA.MC,ANE.MC,BBVA.MC,BKT.MC,CABK.MC,CLNX.MC,...,NTGY.MC,RED.MC,REP.MC,ROVI.MC,SAB.MC,SAN.MC,SCYR.MC,SLR.MC,TEF.MC,UNI.MC
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-31,18.907900,6.923791,125.800224,49.508205,105.200439,NaN,2.819356,2.486552,1.552799,43.892067,...,15.856415,11.810524,5.992204,37.337135,0.279963,2.022224,1.421884,21.240000,2.414027,0.443355
2021-02-28,18.628546,7.272244,139.239990,54.181984,113.449783,NaN,3.445048,2.976777,1.790890,40.930191,...,15.409966,10.456763,7.683920,41.567448,0.322368,2.426250,1.645232,18.600000,2.395008,0.563705
2021-03-31,20.775166,8.390760,136.670639,56.895802,121.529053,NaN,3.312575,3.208760,1.963506,44.609821,...,16.023500,11.388180,7.790898,42.303162,0.352856,2.425831,1.716012,18.080000,2.592668,0.665809
2021-04-30,19.944450,8.707535,142.995224,53.428150,123.059853,NaN,3.542232,3.429996,1.984339,46.141327,...,16.353176,11.520161,7.332002,44.326351,0.408105,2.715462,1.813530,17.055000,2.617460,0.642792
2021-05-31,18.746166,8.850839,141.809372,58.177322,117.446899,NaN,3.895280,3.530102,2.107914,48.142357,...,16.429840,12.383704,8.059445,52.419128,0.485022,2.891989,1.736459,15.940000,2.730554,0.713691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-30,68.000000,11.100000,23.270000,66.933937,170.899994,22.040001,16.041893,13.113783,8.782380,29.087013,...,25.855890,16.224676,14.608485,58.150002,3.231616,8.760357,3.518647,10.895000,4.216414,2.330000
2025-10-31,71.199997,11.180000,23.549999,65.763832,192.000000,23.860001,17.116915,12.786061,8.994429,26.660631,...,25.680002,15.405546,15.393940,61.099998,3.175797,8.826000,3.769556,14.995000,4.205869,2.340000
2025-11-30,79.650002,12.170000,23.459999,62.808826,172.899994,21.080000,18.565001,13.560000,9.616000,25.506618,...,26.520000,15.178559,15.495758,59.450001,3.072973,9.243000,3.801166,16.665001,3.580788,2.530000


In [ ]:
def compute_clean_returns(px_clean: pd.DataFrame,
                          winsorize: bool = True,
                          p: float = 0.005) -> pd.DataFrame:
    """
    Calcula retornos mensuales y (opcional) winsoriza por columna.
    p=0.005 => recorta 0.5% cola inferior y superior.
    """
    rets = px_clean.pct_change()

    # quitar primera fila (NaN por construcción)
    rets = rets.dropna(how="all")

    if winsorize:
        lo = rets.quantile(p, axis=0)
        hi = rets.quantile(1 - p, axis=0)
        rets = rets.clip(lower=lo, upper=hi, axis=1)

    return rets

rets_clean = compute_clean_returns(px_clean, winsorize=True, p=0.005)

print("Retornos (clean):", rets_clean.shape)
rets_clean.describe().T[["mean", "std", "min", "max"]].head()


Retornos (clean): (60, 34)


,mean,std,min,max
Ticker,,,,
ACS.MC,0.028812,0.053569,-0.107216,0.122556
ACX.MC,0.014179,0.079124,-0.218916,0.187200
AENA.MC,0.002340,0.107799,-0.680967,0.167140
AMS.MC,0.005888,0.062855,-0.143785,0.182199
ANA.MC,0.012680,0.075312,-0.127841,0.166207


## 2) Definimos la matriz X

La matriz $X$ es $N\times K$ donde N es el número de activos y $K$ el número de vectores.
$$
X=\left[\begin{array}{ccccc}
z_{1,Value} & z_{1,Mom} & \beta_1 &\dots & Sector_M \\
z_{2,Value} & z_{2,Mom} & \beta_2 &\dots & Sector_M\\
\vdots      & \vdots & \vdots & \dots & \vdots \\
z_{N,Value} & z_{N,Mom} & \beta_N &\dots & Sector_M
\end{array}\right]
$$
Donde:
- **$z_{i,k}$** es la medida factor $j$ del activo $i$ al  normalizado.
$$z_{i,k}=\frac{señal_{i,k}-\mu_k}{\sigma_k}$$

  - $\mu_k$ y $\sigma_k$ se calculan en el corte cross-sectional del correspondiente mes.
  - Si $z_{i,k}>0$ es considerado alto con respecto al factor $k$
  - Si $z_{i,k}<0$ es considerado bajo con respecto al factor $k$

- **$Sector_m$** toma valor 1 si la empresa pertenece al sector $m$.



## 2.1. Estimamos la señal momentum

Definimos el momentum 12-2, es decir, los rendimientos acumulados desde $t-12$ hasta $t-2$, excluyendo el último mes $t-1$ y el actual $t$:
$$MOM_{i,t}=\prod_{j=2}^{12}(1+r_{i,t-j})-1$$

In [ ]:
#Estimamos el factor Momentum
# --- 1) MOM 12-2 (a partir de retornos mensuales)
R = rets_clean.copy().dropna(how="all") # por si aún quedan NAN
assets = R.columns # guarda la lista de tickers

# (1+R) acumulado de t-12..t-2 => shift(2) y rolling(11)
mom_12_2 = (1 + R).shift(2).rolling(11).apply(np.prod, raw=True) - 1 # desplazamos dos meses shift(2)
  # .rolling(11) toma una ventana móvil de 11 meses sobre los datos ya desplazados.
  # .apply(np.prod, raw=True) Para cada ventana, multiplica los 11 factores, trabaja con ndarray en lugar de con series
mom_signal = mom_12_2.iloc[-1]  # señal cross-sectional en la última fecha
mom_signal.head()

,2026-01-31
Ticker,
ACS.MC,0.704770
ACX.MC,0.367315
AENA.MC,-0.601660
AMS.MC,-0.052524
ANA.MC,0.645223


## 2.2. Estimamos la señal valor

In [ ]:
# --- 2) VALUE proxy: 1/PB desde Yahoo
pb = {}
for tkr in assets:
    try:
        pb[tkr] = yf.Ticker(tkr).info.get("priceToBook", np.nan)
    except Exception:
        pb[tkr] = np.nan

pb = pd.Series(pb, dtype=float)
value_proxy = (1.0 / pb).rename("ValueProxy")

## 2.3 Estimamos las betas

In [ ]:
# Últimos 5 años (aprox.)
end = dt.datetime.today()
start = end - dt.timedelta(days=5*365)

# Obtenemos los datos del IBEX35
mkt_px = yf.download(
    "^IBEX",
    start=start.strftime("%Y-%m-%d"),
    end=end.strftime("%Y-%m-%d"),
    auto_adjust=True,
    progress=False
)["Close"].dropna()

mkt_m  = mkt_px.resample("ME").last()
mkt_ret = mkt_m.pct_change().dropna()
mkt_ret.name = "MKT"
mkt_ret.head()

Ticker,^IBEX
Date,
2021-02-28,0.060264
2021-03-31,0.043161
2021-04-30,0.027389
2021-05-31,0.037879
2021-06-30,-0.035819


In [ ]:
# --- 3) Beta de mercado

# Alinear ventanas
common = R.index.intersection(mkt_ret.index)
R_al = R.loc[common]
mkt_al = mkt_ret.loc[common]

def estimate_beta(asset_ret: pd.Series, mkt_ret: pd.Series, min_obs=24) -> float:
    df = pd.concat([asset_ret, mkt_ret], axis=1).dropna()
    if len(df) < min_obs:
        return np.nan
    x = df.iloc[:, 1].values
    y = df.iloc[:, 0].values
    x = x - x.mean()
    y = y - y.mean()
    return float((x @ y) / (x @ x))

beta = pd.Series({tkr: estimate_beta(R_al[tkr], mkt_al) for tkr in assets}, name="MktBeta")
beta.head()

,MktBeta
ACS.MC,0.570978
ACX.MC,1.227074
AENA.MC,1.131287
AMS.MC,0.960155
ANA.MC,0.862130


In [ ]:
# --- 4) Ensamblar exposiciones X (Value, Mom, Beta)
signals = pd.concat([mom_signal.rename("MomRaw"), value_proxy,beta], axis=1)

# Filtrar activos sin señal (P/B faltante o momentum NaN)
signals = signals.dropna(subset=["MomRaw", "ValueProxy","MktBeta"]).copy()
signals.head()

,MomRaw,ValueProxy,MktBeta
ACS.MC,0.704770,0.188767,0.570978
ACX.MC,0.367315,0.669382,1.227074
AENA.MC,-0.601660,0.231387,1.131287
AMS.MC,-0.052524,0.189753,0.960155
ANA.MC,0.645223,0.441508,0.862130


## 2.4. Estimamos la matriz X

In [ ]:
# --- 2.4) Matriz X
def zscore(s: pd.Series) -> pd.Series: # Función para estandarizar las señales
    s = s.replace([np.inf, -np.inf], np.nan) # Si en s hay valores +inf o -inf los reemplaza por NaN.
    return (s - s.mean()) / s.std(ddof=0) # ddof=0 sin perder grados de libertad en la estimación de la desviación típica

X = pd.DataFrame(index=signals.index) # El dataframe lo llamamos X
X["Mom"] = zscore(signals["MomRaw"]) # Estimamos las z para los dos factores
X["Value"] = zscore(signals["ValueProxy"])
X["MktBeta"] = signals["MktBeta"]

print("Activos utilizables:", len(X), "de", len(assets))
X.head()

Activos utilizables: 34 de 34


,Mom,Value,MktBeta
ACS.MC,0.581417,-0.961113,0.570978
ACX.MC,-0.138997,0.255106,1.227074
AENA.MC,-2.207607,-0.853260,1.131287
AMS.MC,-1.035287,-0.958618,0.960155
ANA.MC,0.454294,-0.321539,0.862130


# 3. Planteamos la función de optimización
Si $w$ es el vector de peso de los activos, la exposición de la cartera a los factores es:
$$b(w)=X^Tw\in \mathbb{R}^K$$

Cada exposición de cartera es un promedio ponderado:

$$(X^Tw)_k=\sum_{i=1}^Nw_iX_{i,k}$$

Ahora tenemos que conseguir la siguiente optimización:

1. Tracking de exposiciones:
$$\|X^Tw-b^*\|^2_W=(X^Tw-b^*)^TW(X^Tw-b^*)=\sum_{k=1}^KW_k((B^Tw)_k-b^*_k)^2$$

- $b^*$: e s la exposición deseada.
- $W_k$ tiene distintas utilidades $(W_k\geq 0)$:
  - Permite determinar el peso (la **importancia**) que le das al error de exposición del factor $k$ dentro del término de tracking.
    - $W_k$ grande significa “no tolero” desviarme del objetivo en el factor $k$.
    - $W_k$ pequeño significa el factor es secundario; acepto desviación.
  - Permite **normalizar por incertidumbre**: si los factores tienen distintas escalas, es importante normalizar
  $$W_k=\frac{1}{\sigma^2_k}$$
    - donde $\sigma^2_k$ es el error típico de la exposición agregada del factor. Lo podemos estimar de dos maneras diferentes
      -  Dispersión cross-sectional de la exposición del factor. La desviación típica de la columna $k$ de la matriz $X$
      $$W_k=\frac{1}{std(X_{·,k})^2}$$
      - Si lo que tenemos son betas estimadas de un factor $k$ en una regresión podemos usar la varianza de las betas
  - Pertimte determinar la **tolerancia** al error en cada factor $\delta_k$
  $$W_k=\frac{1}{\delta_k^2}$$

   
2. Penalización por riesgo:
$$\lambda w^T\Sigma w=\lambda \sigma^2(r_p)$$

3. Penalización por rotación
$$\tau \|w-w^{prev}\|_2^2=\sum_i(w_i-w_i^{prev})^2$$
    - $\tau$ grande implica cambios suaves, coste de transacción bajos peor ajuste.
    - $\tau$ implica pequeña implica libertad para reconfigurar.

4. Restricciones adicionales
$$\sum_i^Nw_i=1$$
$$0\leq w_i\leq w_{max}$$

## 3.1 Estimamos la matriz de varianzas y covarianzas

In [ ]:
# --- 3.1. Matriz Covarianza de la rentabilidad de los activos Sigma (solo sobre activos disponibles en X)
R_use = R[X.index].dropna(how="all") # Elimina las filas con NaN
Sigma = R_use.cov().values # Estima la covarianza
Sigma = Sigma + 1e-6*np.eye(Sigma.shape[0])  # regularización suave

In [ ]:
# Definimos la función de optimización
def topdown_cvxpy(
    X: pd.DataFrame, # las características de los activos
    Sigma: np.ndarray, # la matriz de covarianzas
    targets: pd.Series, # la definición de los factores de la cartera
    w_prev: pd.Series | None = None, # cartera de partida si la tenemos
    max_weight: float = 1, # para limitar el peso de los activos
    risk_aversion: float = 1.0, # aversión al riesgo
    turnover_weight: float = 0.0, # nivel de rotación permitida si tenemos una cartera previa
    exp_weights: pd.Series | None = None, # Pesos de W_k para determinar la importancia de los factores
    solver: str = "OSQP", # optimazador
) -> pd.Series:
    """
    min  sum_k a_k ( (B'w - b*)_k )^2 + risk_aversion * w' Σ w + turnover_weight * ||w-w_prev||^2
    s.t. sum(w)=1, 0<=w<=max_weight
    """
    assets = X.index # identificamos los activos
    N = len(assets) # número de activos

    cols = targets.index.intersection(X.columns) # si nuestra definición de factores coincide con los factores tipificados para cada acción
    if len(cols) == 0:
        raise ValueError("Targets no coincide con columnas de X.")

    B = X[cols].values          # N x K, caraterísticas de los activos
    b = targets.loc[cols].values # Definición de los objetivos que queremos obtener

    if exp_weights is None:
        a = np.ones(len(cols)) # matriz de 1
    else:
        a = exp_weights.reindex(cols).fillna(1.0).values #incluimos la definición de W_k

    if w_prev is None:
        w_prev_vec = np.zeros(N) #matriz de ceros
    else:
        w_prev_vec = w_prev.reindex(assets).fillna(0.0).values # incluimos las ponderaciones de la cartera de partida

    w = cp.Variable(N) # definimos el vector de ponderaciones

    exp_err = B.T @ w - b # definimos el  error
    exp_loss = cp.sum(cp.multiply(a, cp.square(exp_err))) # a recoge la importancia de cada factor
    risk_loss = cp.quad_form(w, Sigma) # estima la varianza de la cartera
    turn_loss = cp.sum_squares(w - w_prev_vec) # estima el coste de adaptar la cartera

    obj = cp.Minimize(exp_loss + risk_aversion * risk_loss + turnover_weight * turn_loss) # función objetivo
    cons = [cp.sum(w) == 1.0, w >= 0.0, w <= max_weight] # Restricción

    prob = cp.Problem(obj, cons)
    prob.solve(solver=solver, verbose=False)

    if w.value is None:
        raise RuntimeError(f"Optimización fallida. Status={prob.status}")

    return pd.Series(np.array(w.value).ravel(), index=assets, name="weight")


       target  achieved
Value     0.4  0.398809
Mom       0.3  0.299114

Suma pesos: 1.0 | max: 0.06034836275213905 | min: 3.6557371688938615e-24


,weight
MTS.MC,0.060348
COL.MC,0.059133
UNI.MC,0.053686
REP.MC,0.048682
SAB.MC,0.043678
MRL.MC,0.043612
SAN.MC,0.043494
IDR.MC,0.041057
MAP.MC,0.038928
BBVA.MC,0.038040


In [ ]:
# --- Targets top-down del ejemplo 1. Sin incluir la beta de mercado
targets = pd.Series({"Value": 0.40, "Mom": 0.30})

# Pesos por exposición: priorizar cumplir Value/Mom
exp_weights = pd.Series({"Value": 10.0, "Mom": 10.0})

# (opcional) cartera previa: equal-weight para penalizar rotación
w_prev = pd.Series(1.0/len(X), index=X.index)

w_opt = topdown_cvxpy(
    X=X,
    Sigma=Sigma,
    targets=targets,
    w_prev=w_prev,
    max_weight=0.10,
    risk_aversion=1.0,
    turnover_weight=1.0,   # sube si quieres más estabilidad
    exp_weights=exp_weights,
    solver="OSQP"
)

# --- Chequeos
achieved = (X[targets.index].T @ w_opt).rename("achieved") # factores de la cartera
print(pd.concat([targets.rename("target"), achieved], axis=1))
print("\nSuma pesos:", float(w_opt.sum()), "| max:", float(w_opt.max()), "| min:", float(w_opt.min()))

w_opt.sort_values(ascending=False).head(10)


       target  achieved
Value     0.4  0.398809
Mom       0.3  0.299114

Suma pesos: 0.9999999999999996 | max: 0.060436665039017436 | min: -1.7556245173711818e-23


,weight
MTS.MC,0.060437
COL.MC,0.059019
UNI.MC,0.053709
REP.MC,0.048756
SAB.MC,0.043707
MRL.MC,0.043641
SAN.MC,0.043431
IDR.MC,0.041056
MAP.MC,0.038830
BBVA.MC,0.038046


In [ ]:
# Retorno mensual de la cartera (serie)
port_rets = (R_use[w_opt.index] * w_opt).sum(axis=1)

print("Media mensual:", port_rets.mean(), " | Vol mensual:", port_rets.std())
port_rets.tail()


Media mensual: 0.019228834021130094  | Vol mensual: 0.04404250680526641


,0
Date,
2025-09-30,0.020204
2025-10-31,0.047030
2025-11-30,0.014429
2025-12-31,0.041532
2026-01-31,0.027059


In [ ]:
def topdown_cvxpy(
    X: pd.DataFrame,
    Sigma: np.ndarray,
    targets: pd.Series,
    w_prev: pd.Series | None = None,
    max_weight: float = 1,
    risk_aversion: float = 1.0,
    turnover_weight: float = 0.0,
    exp_weights: pd.Series | None = None,
    solver: str = "OSQP",
    beta_vec: pd.Series | None = None,
    beta_target: float | None = None,
    beta_band: float | None = None,   # si quieres banda en lugar de igualdad exacta
) -> pd.Series:

    assets = X.index
    N = len(assets)

    cols = targets.index.intersection(X.columns)
    if len(cols) == 0:
        raise ValueError("Targets no coincide con columnas de X.")

    B = X[cols].values
    b = targets.loc[cols].values

    a = np.ones(len(cols)) if exp_weights is None else exp_weights.reindex(cols).fillna(1.0).values
    w_prev_vec = np.zeros(N) if w_prev is None else w_prev.reindex(assets).fillna(0.0).values

    w = cp.Variable(N)

    exp_err  = B.T @ w - b
    exp_loss = cp.sum(cp.multiply(a, cp.square(exp_err)))
    risk_loss = cp.quad_form(w, Sigma)
    turn_loss = cp.sum_squares(w - w_prev_vec)

    cons = [cp.sum(w) == 1.0, w >= 0.0, w <= max_weight]

    # --- Neutralidad a mercado: beta' w = beta_target (o en banda)
    if beta_vec is not None and beta_target is not None:
        beta_aligned = beta_vec.reindex(assets).astype(float).values # garantiza que coincidan los ticker de las betas y de los activos
        if beta_band is None:
            cons.append(beta_aligned @ w == beta_target)
        else:
            cons += [
                beta_aligned @ w >= beta_target - beta_band,
                beta_aligned @ w <= beta_target + beta_band
            ]

    obj = cp.Minimize(exp_loss + risk_aversion*risk_loss + turnover_weight*turn_loss)
    prob = cp.Problem(obj, cons)
    prob.solve(solver=solver, verbose=False)

    if w.value is None:
        raise RuntimeError(f"Optimización fallida. Status={prob.status}")

    return pd.Series(np.array(w.value).ravel(), index=assets, name="weight")


In [ ]:
beta_target = 1         # market-neutral
# beta_target = float(beta_vec.reindex(X.index) @ w_prev)  # benchmark-neutral

w_opt = topdown_cvxpy(
    X=X,
    Sigma=Sigma,
    targets=targets,
    w_prev=w_prev,
    max_weight=0.10,
    risk_aversion=1.0,
    turnover_weight=1.0,
    exp_weights=exp_weights,
    solver="OSQP",
    beta_vec=beta,
    beta_target=beta_target,
    beta_band=0.02          # recomendable: banda para evitar infeasible
)

# --- Chequeos
achieved = (X[targets.index].T @ w_opt).rename("achieved") # factores de la cartera
print(pd.concat([targets.rename("target"), achieved], axis=1))
print("\nSuma pesos:", float(w_opt.sum()), "| max:", float(w_opt.max()), "| min:", float(w_opt.min()))

w_opt.sort_values(ascending=False).head(10)


# check beta
beta_port = float(beta.reindex(w_opt.index).values @ w_opt.values)
print("Beta cartera:", beta_port)


       target  achieved
Value     0.4  0.398837
Mom       0.3  0.299130

Suma pesos: 1.0 | max: 0.06169121972004021 | min: 0.0011552024044811369
Beta cartera: 0.98
